<a href="https://colab.research.google.com/github/karandeep7/Image-Classification-of-Plant-Diseases-Using-CNN/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fetching the data from source

###Download datasets from kaggle

In [ ]:
!kaggle datasets download -d rizwan123456789/potato-disease-leaf-datasetpld
!kaggle datasets download -d adilmubashirchaudhry/plant-village-dataset
!kaggle datasets download -d mexwell/crop-diseases-classification


Dataset URL: https://www.kaggle.com/datasets/rizwan123456789/potato-disease-leaf-datasetpld
License(s): DbCL-1.0
 96% 36.0M/37.4M [00:01<00:00, 38.2MB/s]
100% 37.4M/37.4M [00:01<00:00, 25.9MB/s]
Dataset URL: https://www.kaggle.com/datasets/adilmubashirchaudhry/plant-village-dataset
License(s): CC-BY-NC-SA-4.0
100% 985M/989M [00:30<00:00, 35.3MB/s]
100% 989M/989M [00:30<00:00, 34.1MB/s]
Dataset URL: https://www.kaggle.com/datasets/mexwell/crop-diseases-classification
License(s): other
100% 2.00G/2.00G [01:10<00:00, 38.9MB/s]
100% 2.00G/2.00G [01:10<00:00, 30.3MB/s]


###Import required libraries

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import pandas as pd
import os
from PIL import Image
import json
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from torch import tensor
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models import vgg16
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns

from sklearn.manifold import TSNE
import numpy as np
import gdown


###Unzip the datasets

In [ ]:
import zipfile
with zipfile.ZipFile('/content/crop-diseases-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/crop-diseases-classification')

with zipfile.ZipFile('/content/plant-village-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/plant-village-dataset')

with zipfile.ZipFile('/content/potato-disease-leaf-datasetpld.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/potato-disease-leaf-datasetpld')




###Importing trained models from drive

In [ ]:
file_id = "1uWr7eJ6XuQhvks8tvymCy_kRjaGVPBFf"  # Replace with your file ID
output = "trained_models.zip"  # Specify the desired output filename
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)

with zipfile.ZipFile('/content/trained_models.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/trained_models')

Downloading...
From (original): https://drive.google.com/uc?id=1uWr7eJ6XuQhvks8tvymCy_kRjaGVPBFf
From (redirected): https://drive.google.com/uc?id=1uWr7eJ6XuQhvks8tvymCy_kRjaGVPBFf&confirm=t&uuid=3f9cf948-cebe-4a8c-a32b-c9219dbb63f7
To: /content/trained_models.zip
100%|██████████| 1.74G/1.74G [00:27<00:00, 62.6MB/s]
